In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.estimator import SKLearn
import boto3
import numpy as np
import json

In [ ]:
my_region = boto3.session.Session().region_name

In [ ]:

sagemaker_session = sagemaker.Session()

In [ ]:
# Detect the IAM role assigned to this sagemaker instance, to perform operations on the modules 
# such as inferential and Training modules.
role = get_execution_role()

In [ ]:
## Uploading data to S3
data = sagemaker_session.upload_data('data')

In [ ]:
# module containing script for the classifier and the specified data for training the algorithms
# The dataset is a processed file (cleaned and wrangled dataset).
model_path = "production_model.py"

# instantiating the SKLearn estimator.
sklearn_estimator = SKLearn(
    entry_point = model_path,
    train_instance_type = "ml.m4.xlarge",
    role=role,
    sagemaker_session = sagemaker_session,
    framework_version='0.23-1' # Latest version of sklearn at the time of creating this estimator...
)

In [ ]:
# Fitting the estimator with the dataset uploaded to S3.
# This function genereate a training job which the inferential module picks and creates a model with.
sklearn_estimator.fit({'train': data})

In [ ]:
# Model deployment and Endpoint creation.
deployment = sklearn_estimator.deploy(initial_instance_count=1,instance_type="ml.m4.xlarge")

In [ ]:
deployment.endpoint